In [4]:
!huggingface-cli whoami

bcb593


In [5]:
from transformers import AutoTokenizer
import transformers
import torch

In [6]:

model = "meta-llama/Llama-2-7b-chat-hf" # meta-llama/Llama-2-7b-hf

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model, use_auth_token=True)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:656: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [8]:
from transformers import pipeline

llama_pipeline = pipeline(
    "text-generation",  # LLM task
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

2023-10-19 10:40:46.349273: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
'''

import torch
import transformers
from torch import cuda, bfloat16

model_id = 'meta-llama/Llama-2-7b-chat-hf'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'
device_name = torch.cuda.get_device_name()
print(f"Using device: {device} ({device_name})")

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

hf_auth = <Copy&paste your User Access Token from Hugging Face>
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=hf_auth
)
model.eval()
print(f"Model loaded on {device}")

tokenizer = transformers.AutoTokenizer.from_pretrained(…






'''

'\n\nimport torch\nimport transformers\nfrom torch import cuda, bfloat16\n\nmodel_id = \'meta-llama/Llama-2-7b-chat-hf\'\n\ndevice = f\'cuda:{cuda.current_device()}\' if cuda.is_available() else \'cpu\'\ndevice_name = torch.cuda.get_device_name()\nprint(f"Using device: {device} ({device_name})")\n\nbnb_config = transformers.BitsAndBytesConfig(\n    load_in_4bit=True,\n    bnb_4bit_quant_type=\'nf4\',\n    bnb_4bit_use_double_quant=True,\n    bnb_4bit_compute_dtype=bfloat16\n)\n\nhf_auth = <Copy&paste your User Access Token from Hugging Face>\nmodel_config = transformers.AutoConfig.from_pretrained(\n    model_id,\n    use_auth_token=hf_auth\n)\n\nmodel = transformers.AutoModelForCausalLM.from_pretrained(\n    model_id,\n    trust_remote_code=True,\n    config=model_config,\n    quantization_config=bnb_config,\n    device_map=\'auto\',\n    use_auth_token=hf_auth\n)\nmodel.eval()\nprint(f"Model loaded on {device}")\n\ntokenizer = transformers.AutoTokenizer.from_pretrained(…\n\n\n\n\n\n\n

In [10]:
def get_llama_response(prompt: str) -> None:
    """
    Generate a response from the Llama model.

    Parameters:
        prompt (str): The user's input/question for the model.

    Returns:
        None: Prints the model's response.
    """
    sequences = llama_pipeline(
        prompt,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=256,
    )
    print("Chatbot:", sequences[0]['generated_text'])


In [11]:
prompt = 'I liked "Breaking Bad" and "Band of Brothers". Do you have any recommendations of other shows I might like?\n'
get_llama_response(prompt)

RuntimeError: "addmm_impl_cpu_" not implemented for 'Half'

In [ ]:
import torch
from transformers import BertTokenizer, BertForMaskedLM, LineByLineTextDataset, DataCollatorForLanguageModeling, TrainingArguments, Trainer

# 1. Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# 2. Create a toy dataset
with open('toy_dataset.txt', 'w') as f:
    f.write("Hello, world!\n")
    f.write("How are you today?\n")
    f.write("This is a simple dataset for MLM pretraining.\n")
    f.write("Transformers library makes this process easy.\n")

# 3. Tokenize the dataset
dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="toy_dataset.txt",
    block_size=128
)

# 4. Create a data collator for MLM
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,
    mlm_probability=0.15
)

# 5. Initialize the model
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

# 6. Define training arguments and initialize Trainer
training_args = TrainingArguments(
    output_dir="./mlm_model",
    overwrite_output_dir=True,
    num_train_epochs=1,      # Use more epochs for actual pretraining
    per_device_train_batch_size=8,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

# 7. Train the model
trainer.train()

# 8. Save the model
model.save_pretrained("./mlm_model")
tokenizer.save_pretrained("./mlm_model")
